### XBRL GAAP Facts
**A single balance sheet item in an SEC filing is stored using:**
* **XBRL** - (eXtensible Business Reporting Language) is a markup language for exchanging business information in a computer-readable format and is used to standardize the way that financial data is reported and shared. One filing's XBRL typically contains many documents. It is not uncommon for an average filing to contain hundreds of documents.   
* **XBRL instance document** - The core document in an SEC filing's XBRL data. It contains the actual reported financial data for the entity making the filing. An instance document consists of a series of **facts**, each of which represents a specific disclosure—such as a financial statement line item—taken from the financial reports.
* **XML** - The SEC requires that the XBRL instance document be in XML (eXtensible Markup Language) providing universality, flexibility, and readability across platforms and geographies.     
* **Taxonomy** -  Each fact is tagged with a specific XBRL element (taken from the relevant taxonomy) that defines what the fact represents. The taxonomy plays a crucial role in providing a standardized definition and structure to the financial data being reported providing: concept definitions, standard structure, cross-filing comparability, and standardized concept reporting guidelines.
* **US GAAP Financial Reporting Taxonomy** - The SEC uses this specific taxonomy which is aligned with Generally Accepted Accounting Principles (GAAP) used in the United States. Other jurisdictions may use different taxonomies aligned with their specific accounting standards. In addition, a majority of publicly traded companies also use custom taxonomies to define their own unique balance sheets and other relevant corporate data. 
* **Context** - Each fact or data point is associated with a specific context that includes, among other things, a period of time. This period can be one of two types: a **duration** or an **instant**.
* **Fiscal Duration** - A fact with a duration period is one that applies over a span of time. For example, if a company is reporting its total revenue for the fiscal year 2023, the fact would be associated with a duration that starts on January 1, 2023, and ends on December 31, 2023. Durations are typically used for items on the income statement, cash flow statement, and the statement of changes in equity.
* **Fiscal Instant** - A fact with an instant period applies at a specific point in time. For example, if a company is reporting its total assets as of the end of the fiscal year 2023, the fact would be associated with an instant that is December 31, 2023. Instants are typically used for items on the balance sheet.
* **SEC Reporting Guidelines** - The SEC's rules specify when each type of period should be used. For example, facts representing balance sheet line items should have an instant period, while facts representing income statement line items should have a duration period. These rules help ensure that the XBRL data is consistent and comparable across different companies. 

### One Example Fact
**The JSON output below illustrates the value and metadata for a single fact:** 
* An individual filing's XBRL contains hundreds of relevant facts across multiple taxonomies.
* Each fact may contain any number of supporting properties or nested objects.  
* The actual piece of data provided in each fact is labeled 'value'.
* All of the other information you see is metadata that is critical to understanding the what the 'value' actually is.

In [69]:
get_json(obj)

{'concept': {'xml_id': 'dei_EntityCentralIndexKey',
  'schema_url': 'https://xbrl.sec.gov/dei/2022/dei-2022.xsd',
  'name': 'EntityCentralIndexKey',
  'substitution_group': 'item',
  'concept_type': None,
  'abstract': False,
  'nillable': True,
  'period_type': 'duration',
  'balance': None,
  'labels': [{'label': 'lab_dei_EntityCentralIndexKey',
    'language': 'en-US',
    'text': 'Entity Central Index Key',
    'label_type': 'http://www.xbrl.org/2003/role/verboseLabel'},
   {'label': 'lab_dei_EntityCentralIndexKey',
    'language': 'en-US',
    'text': 'Entity Central Index Key',
    'label_type': 'http://www.xbrl.org/2003/role/label'}],
  'type': 'dei:centralIndexKeyItemType'},
 'context': {'xml_id': 'i824606a34d09452696ab35368f66cff9_D20230103-20230403',
  'entity': '0001723580',
  'segments': [],
  'start_date': '2023-01-03',
  'end_date': '2023-04-03'},
 'value': '0001723580',
 'xml_id': 'id3VybDovL2RvY3MudjEvZG9jOmFiNjQyZmQ5ZjVlMjQxNGU5YjM2NGJlZmNkNjhhMmZhL3NlYzphYjY0MmZkOWY1Z

### GAAP Fact Extraction
**I have created an entire library that can:** 
1. Process, identify, and extract the XBRL instance document from each filing's XBRL document collection.
2. Donwload all relevant taxonomies associated with the filing and cache them. (The SEC will block systems that repeately download the same taxonomies, and each download counts against usage limits.) 
3. Locate all facts embedded in each filing. 
4. Recursively inspect each XBRL fact unpacking all fact contents (properties and objects) and flattening them into output containing one row per fact that includes all relevant metadata.    

### GAAP Fact metadata for one filing 
* The output below shows all of the fact metadata unpacked for a single filing. 

In [3]:
import pandas as pd
import Source.MonitorFilings.Parameters as params

# Read in all facts extracted for a single filing 
gaap = pd.read_csv(params.GAAP_dir + '916789_0000916789-20-000003.csv')

gaap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   concept_xml_id               512 non-null    object 
 1   concept_schema_url           512 non-null    object 
 2   concept_name                 512 non-null    object 
 3   concept_substitution_group   512 non-null    object 
 4   concept_concept_type         0 non-null      float64
 5   concept_abstract             512 non-null    bool   
 6   concept_nillable             512 non-null    bool   
 7   concept_period_type          512 non-null    object 
 8   concept_balance              359 non-null    object 
 9   concept_labels_0_label       512 non-null    object 
 10  concept_labels_0_language    512 non-null    object 
 11  concept_labels_0_text        512 non-null    object 
 12  concept_labels_0_label_type  512 non-null    object 
 13  concept_labels_1_lab

### GAAP concept data flattened to one row per fact
* This output shows a subset of metadata for each row's fact.
* The first column shows the names for each piece of data disclosed in the filing. 
* **Important** - Multiple facts for each item or concept_name may be disclosed in a single filing.  These facts typically cover different fiscal durations using the context_start_date and context_end_date OR different fiscal instants using the context_instant_date    

In [4]:
gaap_cols = ['concept_name','concept_period_type','concept_balance',
     'context_xml_id','context_start_date','context_end_date',
     'context_instant_date','value']

with pd.option_context('display.max_rows', 1000, 'display.max_columns', None): 
    display(gaap[gaap_cols])

,concept_name,concept_period_type,concept_balance,context_xml_id,context_start_date,context_end_date,context_instant_date,value
0,AmendmentFlag,duration,NaN,FD2020Q3YTD,2019-03-01,2019-11-30,NaN,false
1,CurrentFiscalYearEndDate,duration,NaN,FD2020Q3YTD,2019-03-01,2019-11-30,NaN,--02-28
2,DocumentFiscalPeriodFocus,duration,NaN,FD2020Q3YTD,2019-03-01,2019-11-30,NaN,Q3
3,DocumentFiscalYearFocus,duration,NaN,FD2020Q3YTD,2019-03-01,2019-11-30,NaN,2020
4,EntityCentralIndexKey,duration,NaN,FD2020Q3YTD,2019-03-01,2019-11-30,NaN,0000916789
5,AllowanceForDoubtfulReceivablesAndReturns,instant,credit,FI2019Q4,NaN,NaN,2019-02-28,2032000.0
6,AllowanceForDoubtfulReceivablesAndReturns,instant,credit,FI2020Q3,NaN,NaN,2019-11-30,1486000.0
7,OtherAssetsNoncurrentAccumulatedAmortization,instant,credit,FI2019Q4,NaN,NaN,2019-02-28,2115000.0
8,OtherAssetsNoncurrentAccumulatedAmortization,instant,credit,FI2020Q3,NaN,NaN,2019-11-30,2167000.0
9,CommonStockParOrStatedValuePerShare,instant,NaN,FI2019Q4,NaN,NaN,2019-02-28,0.1


### GAAP Period Considerations

The most common instances and durations disclosed in SEC filings largely correspond to the primary types of financial statements companies are required to provide:

 1. **Balance Sheet (Statement of Financial Position):** This provides a snapshot of a company's financial position at a specific point in time, typically the end of the fiscal year or quarter. As such, it uses an "instant" context for facts disclosed, like total assets, total liabilities, and equity.
 2. **Income Statement (Statement of Comprehensive Income):** This covers a "duration" period, usually the fiscal year or quarter, during which the company's revenue, costs, profits, and losses occur. Facts disclosed under a duration context might include revenue, cost of goods sold, operating expenses, and net income. 
 3. **Statement of Cash Flows:** This also covers a "duration" period, describing the inflows and outflows of cash during a fiscal year or quarter in operating, investing, and financing activities.
 4. **Statement of Changes in Equity:** This shows changes in equity over a "duration" period and may include items such as dividends, stock issuance, or changes due to net income.
 5. **Notes to Financial Statements:** This part of the filing can have both "instant" and "duration" disclosures, depending on the nature of the data disclosed. This could range from detailed breakdowns of assets and liabilities (which might use "instant" context) to data about the company's revenue recognition policies or lease obligations (which might use "duration" context).

The most common periods for SEC filings would be three months for the quarterly filings and one year for the annual filings with the most common instance dates being the last day of each fiscal quarter and the last day of the fiscal year.

### View the Raw GAAP Data 
* Raw GAAP data is extracted from the XBRL instance document.  This document is one of possibly hundreds of documents embedded in the filing's xbrl.  
* The instance document may also be seen on the SEC's website for most filings when it is provided and can be located. 
* See the last document entry number 33 with the description "EXTRACTED XBRL INSTANCE DOCUMENT" in the following [sec website example](https://www.sec.gov/Archives/edgar/data/1066719/0001683168-23-003732-index.html).   

In [6]:
from Source.GAAP.GaapFeatures import load_raw_gaap_data

gaap_raw = load_raw_gaap_data('916789_0000916789-20-000003.csv')

with pd.option_context('display.max_rows', 1000, 'display.max_columns', None): 
    display(gaap_raw)

,concept_xml_id,concept_schema_url,concept_name,concept_substitution_group,concept_concept_type,concept_abstract,concept_nillable,concept_period_type,concept_balance,concept_labels_0_label,concept_labels_0_language,concept_labels_0_text,concept_labels_0_label_type,concept_labels_1_label,concept_labels_1_language,concept_labels_1_text,concept_labels_1_label_type,concept_type,context_xml_id,context_entity,context_start_date,context_end_date,value,xml_id,footnote,concept_labels_2_label,concept_labels_2_language,concept_labels_2_text,concept_labels_2_label_type,context_instant_date,unit_unit_id,unit_unit,decimals,unit_numerator,unit_denominator
0,dei_AmendmentFlag,https://xbrl.sec.gov/dei/2019/dei-2019-01-31.xsd,AmendmentFlag,item,NaN,False,True,duration,NaN,lab_dei_AmendmentFlag_5D284E133EF572B0C51CDC8E...,en-US,Amendment Flag,http://www.xbrl.org/2003/role/terseLabel,lab_dei_AmendmentFlag_5D284E133EF572B0C51CDC8E...,en-US,Amendment Flag,http://www.xbrl.org/2003/role/label,xbrli:booleanItemType,FD2020Q3YTD,916789,2019-03-01,2019-11-30,false,Fact-87588C95AF19570BA20A05231048F4A7-wk-Fact-...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,dei_CurrentFiscalYearEndDate,https://xbrl.sec.gov/dei/2019/dei-2019-01-31.xsd,CurrentFiscalYearEndDate,item,NaN,False,True,duration,NaN,lab_dei_CurrentFiscalYearEndDate_4345D9F5457CA...,en-US,Current Fiscal Year End Date,http://www.xbrl.org/2003/role/terseLabel,lab_dei_CurrentFiscalYearEndDate_4345D9F5457CA...,en-US,Current Fiscal Year End Date,http://www.xbrl.org/2003/role/label,xbrli:gMonthDayItemType,FD2020Q3YTD,916789,2019-03-01,2019-11-30,--02-28,Fact-93FBA869F0515044BCA841D8B9ADA0DE-wk-Fact-...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,dei_DocumentFiscalPeriodFocus,https://xbrl.sec.gov/dei/2019/dei-2019-01-31.xsd,DocumentFiscalPeriodFocus,item,NaN,False,True,duration,NaN,lab_dei_DocumentFiscalPeriodFocus_0EE92EC39AC0...,en-US,Document Fiscal Period Focus,http://www.xbrl.org/2003/role/terseLabel,lab_dei_DocumentFiscalPeriodFocus_0EE92EC39AC0...,en-US,Document Fiscal Period Focus,http://www.xbrl.org/2003/role/label,dei:fiscalPeriodItemType,FD2020Q3YTD,916789,2019-03-01,2019-11-30,Q3,Fact-AFA4036E9A9257A9B4359A764409D1AF-wk-Fact-...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,dei_DocumentFiscalYearFocus,https://xbrl.sec.gov/dei/2019/dei-2019-01-31.xsd,DocumentFiscalYearFocus,item,NaN,False,True,duration,NaN,lab_dei_DocumentFiscalYearFocus_D877BA5B88EF98...,en-US,Document Fiscal Year Focus,http://www.xbrl.org/2003/role/terseLabel,lab_dei_DocumentFiscalYearFocus_D877BA5B88EF98...,en-US,Document Fiscal Year Focus,http://www.xbrl.org/2003/role/label,xbrli:gYearItemType,FD2020Q3YTD,916789,2019-03-01,2019-11-30,2020,Fact-4FD82D8CF28E539390915456E914C901-wk-Fact-...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,dei_EntityCentralIndexKey,https://xbrl.sec.gov/dei/2019/dei-2019-01-31.xsd,EntityCentralIndexKey,item,NaN,False,True,duration,NaN,lab_dei_EntityCentralIndexKey_4416755CF51C73A4...,en-US,Entity Central Index Key,http://www.xbrl.org/2003/role/terseLabel,lab_dei_EntityCentralIndexKey_4416755CF51C73A4...,en-US,Entity Central Index Key,http://www.xbrl.org/2003/role/label,dei:centralIndexKeyItemType,FD2020Q3YTD,916789,2019-03-01,2019-11-30,0000916789,Fact-A151E9D698F3599D99D70CABC61652C0-wk-Fact-...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
5,hele_AllowanceForDoubtfulReceivablesAndReturns,https://www.sec.gov/Archives/edgar/data/916789...,AllowanceForDoubtfulReceivablesAndReturns,item,NaN,False,True,instant,credit,lab_hele_AllowanceForDoubtfulReceivablesAndRet...,en-US,"Receivables - principally trade, allowances (i...",http://www.xbrl.org/2003/role/terseLabel,lab_hele_AllowanceForDoubtfulReceivablesAndRet...,en-US,Allowance for Doubtful Receivables and Returns,http://www.xbrl.org/2003/role/label,xbrli:monetaryItemType,FI2019Q4,916789,NaT,NaT,2032000.0,Fact-26B25EF56B6B5D4CB66183C9A2A59756-wk-Fact-...,NaN,lab_hele_AllowanceForDoubtfulReceivablesAndRet...,en-US,A valuation allowance principally fo

### View the filing GAAP metadata
* While most XBRL facts contain numeric GAAP data / balance sheet items, many others contain filing specific metadata and filing text items. 
* This code extracts all filing specific metadata. 

In [7]:
from Source.GAAP.GaapFeatures import get_filing_metadata

gaap_meatadata = get_filing_metadata(gaap_raw)
gaap_meatadata

,concept_name,value
0,AmendmentFlag,false
1,CurrentFiscalYearEndDate,--02-28
2,DocumentFiscalPeriodFocus,Q3
22,DocumentType,10-Q
23,DocumentQuarterlyReport,true
24,DocumentPeriodEndDate,2019-11-30
25,DocumentTransitionReport,false
26,EntityFileNumber,001-14669
27,EntityRegistrantName,HELEN OF TROY LIMITED
28,EntityIncorporationStateCountryCode,D0


### What is the approximate 90 day period surrounding the filing?
Here we use the filing specific metadata to determine the fiscal year ending date and the current quarter dates for the filing in process.

In [8]:
from datetime import datetime, timedelta, date, time

gaap_metadata_dict = get_filing_metadata(gaap_raw, as_dict=True)

# Get the DocumentPeriodEndDate reported for this filing 
document_period_end_date = pd.Timestamp(gaap_metadata_dict['DocumentPeriodEndDate'])

# Find a cutoff date for approximately one quarter before the DocumentPeriodEndDate
document_period_start_date = document_period_end_date - timedelta(days=97)

# Print the quarter and approximate dates for this filing 
print('CurrentFiscalYearEndDate:', gaap_metadata_dict['CurrentFiscalYearEndDate'])
print('DocumentFiscalPeriodFocus:', gaap_metadata_dict['DocumentFiscalPeriodFocus'])
print('Quarter Start Date:', document_period_start_date)
print('DocumentPeriodEndDate:', document_period_end_date)

CurrentFiscalYearEndDate: --02-28
DocumentFiscalPeriodFocus: Q3
Quarter Start Date: 2019-08-25 00:00:00
DocumentPeriodEndDate: 2019-11-30 00:00:00


### View All Fiscal Durations in this GAAP Data
* This function identifies all fiscal durations disclosed in a filing's XBRL instance document facts.
* The data is pivoted to create one row per duration containing all numeric facts / features disclosed in the filing. 
* Text / String and html concept types are excluded to avoid including items like a zip code that could possibly be converted to a numeric GAAP feature.   

In [9]:
from Source.GAAP.GaapFeatures import get_fiscal_duration_features

gaap_FD = get_fiscal_duration_features(gaap_raw)

gaap_FD

concept_name,context_start_date,context_end_date,AllocatedShareBasedCompensationExpenseNetOfTax,AmortizationOfFinancingCosts,AmortizationOfIntangibleAssets,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect,CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations,CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations,CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations,ComprehensiveIncomeNetOfTax,...,StockIssuedDuringPeriodValueShareBasedCompensationExcludingStockOptionsAndEmployeeStockPurchaseProgramNetOfTaxEffect,StockIssuedDuringPeriodValueStockOptionsExercisedNetOfTaxEffect,StockRepurchasedAndRetiredDuringPeriodValue,UnrecognizedTaxBenefitsDecreasesResultingFromRecognitionOfExcessTaxBenefitsFromShareBasedCompensation,Unrecognizedtaxbenefitfromfavorableresolutionofuncertaintaxposition,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic,duration_days,duration_YTD_days_dist,duration_90_days_dist
0,2018-03-01,2018-11-30,16020000.0,761000.0,10822000.0,-1602000.0,0.0,0.0,-5231000.0,135855000.0,...,17029000.0,6081000.0,142415000.0,700000.0,800000.0,26520000.0,26321000.0,274 days,4 days,184 days
1,2018-09-01,2018-11-30,5618000.0,NaN,3300000.0,NaN,NaN,NaN,NaN,50030000.0,...,6016000.0,566000.0,100175000.0,NaN,NaN,26366000.0,26057000.0,90 days,180 days,0 days
2,2019-03-01,2019-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days,270 days,90 days
3,2019-03-01,2019-11-30,17309000.0,763000.0,13129000.0,7766000.0,0.0,0.0,0.0,149412000.0,...,18743000.0,4190000.0,10133000.0,1000000.0,2800000.0,25295000.0,25099000.0,274 days,4 days,184 days
4,2019-05-08,2019-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days,270 days,90 days
5,2019-09-01,2019-11-30,4415000.0,NaN,4790000.0,NaN,NaN,NaN,NaN,68569000.0,...,4758000.0,557000.0,1002000.0,NaN,NaN,25396000.0,25161000.0,90 days,180 days,0 days


### Get the two most recent QoQ (quarter over quarter) fiscal durations 
**Here are the current rules I am using to identify a filng's current QoQ fiscal durations:** 
* The fiscal duration periods closest to 90 days are considered quarterly durations.
* Fiscal durations must also be within 7 days of the 90 day period to qualify as 'quarterly'. 
* There must be at least two quarterly durations disclosed with the same number of days in each duration. 
* When there are more than two quarterly durations disclosed in a single filing, the two most recent durations are returned. 
* All QoQ raw GAAP features and values are returned in ascending order by context start and end date. 

In [118]:
current_QoQ = get_current_QoQ_fiscal_durations(gaap_FD)

with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
    display(current_QoQ)

concept_name,context_start_date,context_end_date,AllocatedShareBasedCompensationExpenseNetOfTax,AmortizationOfIntangibleAssets,ComprehensiveIncomeNetOfTax,CostOfGoodsAndServicesSold,DepreciationDepletionAndAmortization,EarningsPerShareBasic,EarningsPerShareDiluted,GrossProfit,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeLossFromContinuingOperationsIncludingPortionAttributableToNoncontrollingInterest,IncomeLossFromContinuingOperationsPerBasicShare,IncomeLossFromContinuingOperationsPerDilutedShare,IncomeLossFromDiscontinuedOperationsNetOfTax,IncomeLossFromDiscontinuedOperationsNetOfTaxPerBasicShare,IncomeLossFromDiscontinuedOperationsNetOfTaxPerDilutedShare,IncomeTaxExpenseBenefit,IncomeTaxExpenseBenefitPercentageOfIncomeBeforeIncomeTaxes,IncrementalCommonSharesAttributableToShareBasedPaymentArrangements,InterestExpense,NetIncomeLoss,NetIncomeLossAvailableToCommonStockholdersBasic,NetIncomeLossFromContinuingOperationsAvailableToCommonShareholdersBasic,NonoperatingIncomeExpense,OperatingIncomeLoss,OtherComprehensiveIncomeForeignCurrencyTransactionAndTranslationGainLossArisingDuringPeriodNetOfTax,OtherComprehensiveIncomeLossNetOfTax,PaymentsToAcquireProductiveAssets,RestructuringCharges,RevenueFromContractWithCustomerExcludingAssessedTax,SellingGeneralAndAdministrativeExpense,ShareBasedCompensationEffectOnEarningsPerShareBasic,ShareBasedCompensationEffectOnEarningsPerShareDiluted,StockIssuedDuringPeriodValueEmployeeStockPurchasePlan,StockIssuedDuringPeriodValueRestrictedStockAwardNetOfForfeitures,StockIssuedDuringPeriodValueShareBasedCompensationExcludingStockOptionsAndEmployeeStockPurchaseProgramNetOfTaxEffect,StockIssuedDuringPeriodValueStockOptionsExercisedNetOfTaxEffect,StockRepurchasedAndRetiredDuringPeriodValue,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic,duration_days,duration_YTD_days_dist,duration_90_days_dist
1,2018-09-01,2018-11-30,5618000.0,3300000.0,50030000.0,249236000.0,7195000.0,1.90,1.88,181845000.0,58340000.0,54320000.0,2.08,2.06,-4850000.0,-0.19,-0.18,4020000.0,0.069,309000.0,2971000.0,49470000.0,49470000.0,54320000.0,15000.0,61296000.0,514000.0,560000.0,9105000.0,25000.0,431081000.0,120524000.0,0.22,0.21,1077000.0,0.0,6016000.0,566000.0,100175000.0,26366000.0,26057000.0,90 days,180 days,0 days
5,2019-09-01,2019-11-30,4415000.0,4790000.0,68569000.0,264764000.0,8760000.0,2.73,2.71,209973000.0,76594000.0,68699000.0,2.73,2.71,0.0,0.00,0.00,7895000.0,0.103,235000.0,2767000.0,68699000.0,68699000.0,68699000.0,92000.0,79269000.0,-1729000.0,-130000.0,4386000.0,12000.0,474737000.0,130692000.0,0.18,0.17,1427000.0,0.0,4758000.0,557000.0,1002000.0,25396000.0,25161000.0,90 days,180 days,0 days


## GAAP Feature Engineering for Machine Learning 

### Create QoQ Percentage Change Features for the most recent quarterly fiscal durations
* Here percentage changes for the two most recent quarterly fiscal durations returned above are calculated.
* The suffix '_QoQ' is added to each feature name.  

**Percentage changes create two primary benefits for machine learning in this instance:**
 1. **Scale** - Percentage changes indicate movement across fiscal durations in the exact same scale for all balance sheet items *regardless of market cap, share price, or sector*.
 2. **Imputation** - Most machine learning models do not handle missing values well. Many companies also have custom XBRL taxonomies and business or sector specific items on their balance sheet.  When many filing's GAAP data are combined into training datasets, these unique balance sheet features will become sparse features with many missing values.  Using a percentage indicating 0% change, implies no movement for a missing balance sheet item.  This imputation is more accurate for missing balance sheet items than mean imputation or other strategies which all require making an educated guess for missing values. 

In [120]:
QoQ_features = create_QoQ_pct_chg_features(gaap_FD).T
QoQ_features.columns = ['Pct Change']

print('Total QoQ Percentage Change Features:', len(QoQ_features))
print('Total Non Zero QoQ Percentage Change Features:', len(QoQ_features[QoQ_features['Pct Change'] != 0]))

with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
    display(QoQ_features)

Total QoQ Percentage Change Features: 39
Total Non Zero QoQ Percentage Change Features: 38


,Pct Change
AllocatedShareBasedCompensationExpenseNetOfTax_QoQ,-0.214133
AmortizationOfIntangibleAssets_QoQ,0.451515
ComprehensiveIncomeNetOfTax_QoQ,0.370558
CostOfGoodsAndServicesSold_QoQ,0.062302
DepreciationDepletionAndAmortization_QoQ,0.217512
EarningsPerShareBasic_QoQ,0.436842
EarningsPerShareDiluted_QoQ,0.441489
GrossProfit_QoQ,0.154681
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest_QoQ,0.312890
IncomeLossFromContinuingOperationsIncludingPortionAttributableToNoncontrollingInterest_QoQ,0.264709


### Get the two most recent YTD durations for YoY (year over year) comparisons
* The DocumentFiscalPeriodFocus for this filing is Q3 and comprises 90 * 3 = 270 days of fiscal performance.   
* The fiscal duration periods closest to 270 days are considered YTD durations for all Q3 filings.
* Fiscal durations must also be with 7 days of the 270 day period to qualify as 'YTD'. 
* There must be at least two YTD durations disclosed with the same number of days in each duration. 
* When there are more than two YTD durations disclosed in a single filing, the two most recent durations are returned. 
* All YoY raw GAAP features and values are returned in chronological order. 

In [122]:
with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
    display(get_current_YoY_fiscal_durations(gaap_FD))

concept_name,context_start_date,context_end_date,AllocatedShareBasedCompensationExpenseNetOfTax,AmortizationOfFinancingCosts,AmortizationOfIntangibleAssets,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect,CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations,CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations,CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations,ComprehensiveIncomeNetOfTax,CostOfGoodsAndServicesSold,DeferredIncomeTaxesAndTaxCredits,DepreciationDepletionAndAmortization,EarningsPerShareBasic,EarningsPerShareDiluted,GainLossOnSaleOfPropertyPlantEquipment,GrossProfit,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeLossFromContinuingOperationsIncludingPortionAttributableToNoncontrollingInterest,IncomeLossFromContinuingOperationsPerBasicShare,IncomeLossFromContinuingOperationsPerDilutedShare,IncomeLossFromDiscontinuedOperationsNetOfTax,IncomeLossFromDiscontinuedOperationsNetOfTaxPerBasicShare,IncomeLossFromDiscontinuedOperationsNetOfTaxPerDilutedShare,IncomeTaxExpenseBenefit,IncomeTaxExpenseBenefitPercentageOfIncomeBeforeIncomeTaxes,IncreaseDecreaseInAccountsPayable,IncreaseDecreaseInAccruedIncomeTaxesPayable,IncreaseDecreaseInAccruedLiabilities,IncreaseDecreaseInInventories,IncreaseDecreaseInOtherOperatingCapitalNet,IncreaseDecreaseInPrepaidDeferredExpenseAndOtherAssets,IncreaseDecreaseInReceivables,IncrementalCommonSharesAttributableToShareBasedPaymentArrangements,InterestExpense,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesContinuingOperations,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInInvestingActivitiesContinuingOperations,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInOperatingActivitiesContinuingOperations,NetIncomeLoss,NetIncomeLossAvailableToCommonStockholdersBasic,NetIncomeLossFromContinuingOperationsAvailableToCommonShareholdersBasic,NonoperatingIncomeExpense,OperatingIncomeLoss,OtherComprehensiveIncomeForeignCurrencyTransactionAndTranslationGainLossArisingDuringPeriodNetOfTax,OtherComprehensiveIncomeLossBeforeReclassificationsBeforeTax,OtherComprehensiveIncomeLossBeforeReclassificationsTax,OtherComprehensiveIncomeLossNetOfTax,PaymentsForRepurchaseOfCommonStock,PaymentsToAcquireProductiveAssets,ProceedsFromIssuanceOfSharesUnderIncentiveAndShareBasedCompensationPlansIncludingStockOptionsAndTaxBenefits,ProceedsFromLinesOfCredit,ProceedsFromSaleOfPropertyPlantAndEquipment,ProfitLoss,ProvisionForDoubtfulAccounts,ReclassificationFromAccumulatedOtherComprehensiveIncomeCurrentPeriodBeforeTax,RepaymentsOfLinesOfCredit,RepaymentsOfLongTermDebt,RestructuringCharges,RevenueFromContractWithCustomerExcludingAssessedTax,SellingGeneralAndAdministrativeExpense,ShareBasedCompensation,ShareBasedCompensationEffectOnEarningsPerShareBasic,ShareBasedCompensationEffectOnEarningsPerShareDiluted,StockIssuedDuringPeriodValueEmployeeStockPurchasePlan,StockIssuedDuringPeriodValueRestrictedStockAwardNetOfForfeitures,StockIssuedDuringPeriodValueShareBasedCompensationExcludingStockOptionsAndEmployeeStockPurchaseProgramNetOfTaxEffect,StockIssuedDuringPeriodValueStockOptionsExercisedNetOfTaxEffect,StockRepurchasedAndRetiredDuringPeriodValue,UnrecognizedTaxBenefitsDecreasesResultingFromRecognitionOfExcessTaxBenefitsFromShareBasedCompensation,Unrecognizedtaxbenefitfromfavorableresolutionofuncertaintaxposition,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic,duration_days,duration_YTD_days_dist,duration_90_days_dist
0,2018-03-01,2018-11-30,16020000.0,761000.0,10822000.0,-1602000.0,0.0,0.0,-5231000.0,135855000.0,695732000.0,4060000.0,22490000.0,4.99,4.95,536000.0,483576000.0,147045000.0,136510000.0,5.19,5.15,-5231000.0,-0.2,-0.2,10535000.0,0.072,16694000.0,2969000.0,17809000.0,49137000.0,-5554000.0,1149000.0,64284000.0,199000.0,8413000.0,-84825000.0,-84825000.0,-21041000.0,-2104100

### Create YoY Percentage Change Features for the most recent quarterly fiscal durations
* Here percentage changes for the two most recent YTD fiscal durations returned above are calculated.
* The suffix '_YoY' is added to each feature name.  
* Percentage changes provide the same two primary benefits for machine learning regarding scale and imputation.

In [123]:
YoY_features = create_YoY_pct_chg_features(gaap_FD).T
YoY_features.columns = ['Pct Change']

print('Total YoY Percentage Change Features:', len(YoY_features))
print('Total Non Zero YoY Percentage Change Features:', len(YoY_features[YoY_features['Pct Change'] != 0]))

with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
    display(YoY_features)

Total YoY Percentage Change Features: 73
Total Non Zero YoY Percentage Change Features: 69


,Pct Change
AllocatedShareBasedCompensationExpenseNetOfTax_YoY,0.080462
AmortizationOfFinancingCosts_YoY,0.002628
AmortizationOfIntangibleAssets_YoY,0.213177
CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect_YoY,-5.847690
CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations_YoY,0.000000
CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations_YoY,0.000000
CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations_YoY,-1.000000
ComprehensiveIncomeNetOfTax_YoY,0.099790
CostOfGoodsAndServicesSold_YoY,0.039504
DeferredIncomeTaxesAndTaxCredits_YoY,-1.125862


### View the Fiscal Instance GAAP Data
* This function identifies all fiscal instances disclosed in a filing's XBRL instance document facts.
* The data is pivoted to create one row per instance date containing all numeric facts / features disclosed in the filing.  

In [124]:
gaap_FI = get_fiscal_instance_features(gaap_raw)

with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
    display(gaap_FI)

concept_name,context_instant_date,AccountsPayableCurrent,AccountsReceivableNetCurrent,AccruedAdvertisingCurrent,AccruedLiabilitiesCurrent,AccruedMarketingCostsCurrent,AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,AccumulatedOtherComprehensiveIncomeLossNetOfTax,AdditionalPaidInCapitalCommonStock,AllowanceForDoubtfulReceivablesAndReturns,Assets,AssetsCurrent,CashAndCashEquivalentsAtCarryingValue,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations,CommonStockParOrStatedValuePerShare,CommonStockSharesAuthorized,CommonStockSharesIssued,CommonStockSharesOutstanding,CommonStockValue,DeferredTaxAssetsLiabilitiesNet,DeferredTaxLiabilities,DisposalGroupIncludingDiscontinuedOperationCashAndCashEquivalents,EmployeeRelatedLiabilitiesCurrent,EntityCommonStockSharesOutstanding,FiniteLivedIntangibleAssetsAccumulatedAmortization,FiniteLivedIntangibleAssetsAmortizationExpenseRemainderOfFiscalYear,FiniteLivedIntangibleAssetsAmortizationExpenseYearFive,FiniteLivedIntangibleAssetsAmortizationExpenseYearFour,FiniteLivedIntangibleAssetsAmortizationExpenseYearSix,FiniteLivedIntangibleAssetsAmortizationExpenseYearThree,FiniteLivedIntangibleAssetsAmortizationExpenseYearTwo,Goodwill,GoodwillImpairedAccumulatedImpairmentLoss,IntangibleAssetsGrossIncludingGoodwill,IntangibleAssetsNetExcludingGoodwill,IntangibleAssetsNetIncludingGoodwill,InventoryNet,LesseeOperatingLeaseLiabilityPaymentsDue,LesseeOperatingLeaseLiabilityPaymentsDueAfterYearFive,LesseeOperatingLeaseLiabilityPaymentsDueYearFive,LesseeOperatingLeaseLiabilityPaymentsDueYearFour,LesseeOperatingLeaseLiabilityPaymentsDueYearThree,LesseeOperatingLeaseLiabilityPaymentsDueYearTwo,LesseeOperatingLeaseLiabilityPaymentsRemainderOfFiscalYear,LesseeOperatingLeaseLiabilityUndiscountedExcessAmount,Liabilities,LiabilitiesAndStockholdersEquity,LiabilitiesCurrent,LongTermDebt,LongTermDebtCurrent,LongTermDebtNoncurrent,NewAccountingPronouncementOrChangeInAccountingPrincipleCumulativeEffectOfChangeOnEquityOrNetAssets1,NonCashIncreaseDecreaseInPrepaidExpenseAndOtherAssets,OperatingLeaseLiability,OperatingLeaseLiabilityCurrent,OperatingLeaseLiabilityNoncurrent,OperatingLeaseRightOfUseAsset,OperatingLeaseWeightedAverageDiscountRatePercent,OtherAccruedLiabilitiesCurrent,OtherAssetsNoncurrent,OtherAssetsNoncurrentAccumulatedAmortization,OtherLiabilitiesNoncurrent,PreferredStockParOrStatedValuePerShare,PreferredStockSharesAuthorized,PreferredStockSharesIssued,PreferredStockValue,PrepaidExpenseAndOtherAssetsCurrent,ProductWarrantyAccrualClassifiedCurrent,PropertyPlantAndEquipmentGross,PropertyPlantAndEquipmentNet,RetainedEarningsAccumulatedDeficit,StockRepurchasePriorProgramRemainingAuthorizedRepurchaseAmount,StockRepurchaseProgramAuthorizedAmount1,StockRepurchaseProgramRemainingAuthorizedRepurchaseAmount1,StockholdersEquity,TaxesPayableCurrentAndNoncurrent,context_instant_days_diff,period_end_date_days_diff
0,2018-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20738000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.014459e+09,NaN,0 days,-640 days
1,2018-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-163000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 days,-639 days
2,2018-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

### Important Fiscal Instances

* It appears that there are two primary snapshots provided in the fiscal instance data above:
 * **CurrentFiscalYearEndDate** - Snapshot as of fiscal year end
 * **DocumentPeriodEndDate** - Snapshot as of the document period end

### The Filing Metadata below shows how the primary fiscal instants disclosed align with these dates

In [132]:
# Print important metadata from the filing
print('CurrentFiscalYearEndDate:', gaap_metadata_dict['CurrentFiscalYearEndDate'])
print('DocumentFiscalPeriodFocus:', gaap_metadata_dict['DocumentFiscalPeriodFocus'])
print('DocumentPeriodEndDate:', gaap_metadata_dict['1774'])

CurrentFiscalYearEndDate: --02-28
DocumentFiscalPeriodFocus: Q3
DocumentPeriodEndDate: 2019-11-30


### Create FYED to DPED Percentage Change Features 
* Here percentage changes between the CurrentFiscalYearEndDate and DocumentPeriodEndDate fiscal instant data are calculated.
* The suffix '_FYED_to_DPED' is added to each feature name.  
* Percentage changes provide the same two primary benefits for machine learning regarding scale and imputation.

In [157]:
gaap_metadata_dict = get_filing_metadata(gaap_raw, as_dict=True)
FYED_to_DPED_features = create_FYED_to_DPED_pct_chg_features(gaap_FI, gaap_metadata_dict, feature_suffix='_FYED_to_DPED').T
FYED_to_DPED_features.columns = ['Pct Change']

print('Total FYED to DPED Percentage Change Features:', len(FYED_to_DPED_features))
print('Total Non Zero FYED to DPED Percentage Change Features:', len(FYED_to_DPED_features[FYED_to_DPED_features['Pct Change'] != 0]))

FYED_to_DPED_features

Total FYED to DPED Percentage Change Features: 49
Total Non Zero FYED to DPED Percentage Change Features: 42


,Pct Change
AccountsPayableCurrent_FYED_to_DPED,-0.014189
AccountsReceivableNetCurrent_FYED_to_DPED,0.304207
AccruedAdvertisingCurrent_FYED_to_DPED,0.166522
AccruedLiabilitiesCurrent_FYED_to_DPED,0.036988
AccruedMarketingCostsCurrent_FYED_to_DPED,0.104903
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment_FYED_to_DPED,0.080974
AccumulatedOtherComprehensiveIncomeLossNetOfTax_FYED_to_DPED,-5.101595
AdditionalPaidInCapitalCommonStock_FYED_to_DPED,0.065478
AllowanceForDoubtfulReceivablesAndReturns_FYED_to_DPED,-0.268701
Assets_FYED_to_DPED,0.085814


### GAAP Data Repositories Planned  
**I plan to organize and collect all GAAP data described above into the following data repositories:**
* **10Q_XBRL_Instance** - Contains the extracted XBRL instance document for each 10-Q filing. (This repository is currently populated for all filings.)
* **10Q_GAAP** - One fat file per filing that contains all **facts** extracted from the filing's XBRL instance document. (This repository is currently populated for all filings.)
* **10Q_GAAP_Metadata** - One fat file per filing that contains the filing's extracted metadata.  It is possible that this might be stored as a single pickled dataframe file depending on size and performance. 
* **10Q_GAAP_Periods** - One fat file per filing containing all raw numeric fiscal instance and fiscal duration data extracted from the filing's XBRL instance document. 
* **10Q_GAAP_QoQ** - One fat file per filing containing the most recent raw numeric quarter over quarter fiscal duration features for all filings.
* **10Q_GAAP_YoY** - One fat file per filing containing the most recent raw numeric year over year fiscal duration features for all filings.
* **10Q_GAAP_FYED_to_DPED** - One fat file per filing containing the most recent raw numeric year end over document period end date fiscal instant features for all filings.
* **10Q_GAAP_Percentages** - One record per filing containing all QoQ, YoY, and FYED_to_DPED percentage change features.

These repositories are subject to change as I begin to populate each one and better understand how successful each collection strategy actually is.  